In [44]:
# read data into a DataFrame
import pandas as pd
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
import numpy.random as nprnd
import random
import json
pd.set_option('display.max_columns', 500)
%matplotlib inline

import os
import multiprocessing
import warnings

from tqdm import tqdm
import numpy as np
from scipy import stats
import pandas as pd
import librosa

import utils

tracks = pd.read_csv('tracks.csv',header=None)
genres = pd.read_csv('genres.csv')
#features = pd.read_csv('features.csv')
echonest = pd.read_csv('echonest.csv',header=None)

ModuleNotFoundError: No module named 'tqdm'

## to install package (on command): conda install -c conda-forge librosa

In [ ]:
# Feature extraction example
import numpy as np
import librosa


# Load the example clip
# Load 30 seconds of a wav file, starting 15 seconds in
y, sr = librosa.load('Rough & Laugh.mp3', offset=15.0, duration=30.0)

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms

hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [77]:
##EXAMPLE: echroma_stft 
stft=np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
mean=np.mean(f,axis=1)
sd=np.std(f,axis=1)
skew=stats.skew(f,axis=1)
kurt=stats.kurtosis(f,axis=1)
med=np.median(f,axis=1)
minn=np.min(f,axis=1)
maxx=np.max(f,axis=1)

print('chroma_stft mean=',mean)
print('chroma_stft std=',sd)
print('chroma_stft skew=',skew)
print('chroma_stft kurtosis=',kurt)
print('chroma_stft min=',minn)
print('chroma_stft max=',maxx)

chroma_stft mean= [ 0.226574    0.33784059  0.4044111   0.3343784   0.39587869  0.28776965
  0.43681299  0.44910523  0.44599068  0.55456857  0.32542544  0.34255647]
chroma_stft std= [ 0.20187885  0.27087569  0.32075873  0.24008877  0.31557203  0.23624882
  0.31194482  0.33238854  0.30680735  0.34885626  0.22697222  0.29191849]
chroma_stft skew= [ 1.77601873  1.21638314  0.82716924  0.95373702  0.67288752  1.21007302
  0.50048988  0.43924406  0.31028217  0.01207981  0.82790941  1.01782511]
chroma_stft kurtosis= [ 3.20816663  0.61874171 -0.67225715  0.4475405  -0.77912979  0.93339879
 -0.951679   -1.18391066 -1.16240387 -1.53970165  0.20640102 -0.03881438]
chroma_stft min= [ 0.00369041  0.00558853  0.00293299  0.0070795   0.0037315   0.00307917
  0.00393694  0.0030907   0.00598345  0.01579855  0.00943802  0.00346273]
chroma_stft max= [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [86]:
##Statistic for each feature
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def feature_stats(name, values):
    features[name, 'mean'] = np.mean(values, axis=1)
    features[name, 'std'] = np.std(values, axis=1)
    features[name, 'skew'] = stats.skew(values, axis=1)
    features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
    features[name, 'median'] = np.median(values, axis=1)
    features[name, 'min'] = np.min(values, axis=1)
    features[name, 'max'] = np.max(values, axis=1)


features = pd.Series(index=columns(), dtype=np.float32)


In [94]:
##OTHER FEATURES
y, sr = librosa.load('Rough & Laugh.mp3', offset=15.0, duration=30.0)

stft=np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
feature_stats('chroma_stft', f)

f = librosa.feature.zero_crossing_rate(y, frame_length=2048, hop_length=512)
feature_stats('zcr', f)

cqt = np.abs(librosa.cqt(y, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
assert cqt.shape[0] == 7 * 12
assert np.ceil(len(y)/512) <= cqt.shape[1] <= np.ceil(len(y)/512)+1


f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cqt', f)

f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cens', f)

f = librosa.feature.tonnetz(chroma=f)
feature_stats('tonnetz', f)

print(features)

feature      statistics  number
chroma_cens  kurtosis    01        0.191394
                         02       -0.251092
                         03        0.723342
                         04       -0.027065
                         05       -0.315510
                         06       -0.711926
                         07       -0.868581
                         08       -0.944543
                         09       -0.363966
                         10        0.826686
                         11        0.303585
                         12        1.289770
             max         01        0.438408
                         02        0.500721
                         03        0.665537
                         04        0.502544
                         05        0.463433
                         06        0.472650
                         07        0.606595
                         08        0.591837
                         09        0.573341
                         10        0.623037
